In [1]:
import sys
sys.path.append('D:/study/brec_ijcnn2022')
import numpy as np
import os

In [19]:
path = 'data/data.npz'
data = np.load(path)
x_train_load = data['x_train']
x_test = data['x_test']
y_train_load = data['y_train']
y_test = data['y_test']
print("data loaded from", path)
print(x_train_load.shape)

data loaded from data/data.npz
(19968, 16, 42)


In [21]:
print(y_train_load.shape)
print(y_train_load[0])

(19968, 22)
[0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]


# Preprocess
next step: look at preprocess in details

look up func preprocess() in transformer_model.py
* ncodpers is uid
* y_users is diction, uid as key, items as value, items one one-hot encoded (22)
* 16 is sequence length
* 42 is d_model: to be filled later

idea 1: process y as or relation of its vector?

findings: feasible, not positive samples and negative samples need rebalancing.

problem: labels not as 0 or 1?

In [28]:
# idead 1 experiments
y_summed = np.sum(y_train_load, axis=1)
print(y_summed.shape)
print(y_summed[y_summed==0].shape)
print(y_summed[y_summed>0].shape)

(19968,)
(686,)
(19282,)


In [10]:
from sklearn.preprocessing import MinMaxScaler
def clean_data(df):
    def median_income(df):
        df.loc[df.renta.isnull(), 'renta'] = df.renta.median(skipna=True)
        return df
    # provide median income by province
    df = df.groupby('nomprov').apply(median_income)
    df.loc[df.renta.isnull(), "renta"] = df.renta.median(skipna=True)
    # set entries whose "antiguedad" field is missing as minimum seniority
    df.antiguedad = pd.to_numeric(df.antiguedad, errors="coerce")
    df.loc[df.antiguedad.isnull(), "antiguedad"] = df.antiguedad.min()
    df.loc[df.antiguedad < 0, "antiguedad"] = 0
    df["antiguedad"] = df["antiguedad"].astype(int)
    # fix customers age
    df["age"] = pd.to_numeric(df["age"], errors="coerce")
    df["age"].fillna(df["age"].mean(), inplace=True)
    df["age"] = df["age"].astype(int)
    # fill missing field "segmento" with most frequent one
    df.loc[df["segmento"].isnull(), "segmento"] = "03 - UNIVERSITARIO"
    # normalize scalar columns
    scale_cols = ["antiguedad", "age", "renta"]
    for col in scale_cols:
        scaler = MinMaxScaler()
        df[col] = scaler.fit_transform(df[[col]])
    return df

In [29]:
import pandas as pd
import copy
import math
def preprocess_old(input_file, y_date, seq_len=16, batch_size=32, exclude_date=None, d_model=35):
    """
    Preprocess data and split it in train and test data
    :param d_model:
    :param input_file: string, path to raw dataset, csv file
    :param y_date: string, timestamp use for testing
    :param exclude_date: list[string] timestamps to ignore
    :return: train_x, train_y (both are np.array)
    """
    months_one_hot = [0 for _ in range(12)]
    segmentation_dict = {}
    x_users, y_users = {}, {}
    df = pd.read_csv(input_file)
    df = clean_data(df)
    users = []
    for i, row in df.iterrows():
        if row['fecha_dato'] in exclude_date:
            pass
        user = row['ncodpers'] # uid
        date = row['fecha_dato'].split("-")
        year = [int(date[0] == "2016")]  # 1=2016, 0=2015 (1)
        month = copy.copy(months_one_hot)
        month[int(date[1]) - 1] = 1  # months one-hot encoded (12)
        items = list(row.values)[26:]  # items are one-hot encoded (22)
        items = [int(item) if not math.isnan(item) and item != 'NA' else 0 for item in items]
        # one-hot encode segmentation (4)
        segmentation = row['segmento']
        segmentation_array = [0, 0, 0, 0]
        if segmentation not in segmentation_dict.keys():
            segmentation_dict[segmentation] = len(segmentation_dict)
        segmentation_array[segmentation_dict[segmentation]] = 1
        # one-hot encode new-index (1)
        #new_index = [1] if row['ind_nuevo'] == 1 else [0]
        # seniority + age + income (3) - values features
        seniority = float(row['antiguedad'])
        age = float(row['age'])
        income = float(row['renta'])
        value_features = [seniority, age, income]
        # put the data together
        data = year + month + segmentation_array + value_features + items  # (42) values
        if row['fecha_dato'] == y_date and user in x_users.keys():
            y_users[user] = np.array(items)
            users.append(user)
        elif user in x_users.keys():
            x_users[user] = np.vstack((x_users[user], np.array(data)))
        else:
            x_users[user] = np.array(data)

    assert len(x_users) == len(y_users)
    x_data = []
    y_data = []
    for user in users:
        if np.array(x_users[user]).shape[0] == seq_len:
            x_data.append(x_users[user].reshape((seq_len, d_model)))
        else:
            continue
        y_data.append(y_users[user])
    x_data = np.stack(x_data)
    y_data = np.stack(y_data)
    num_users = x_data.shape[0]
    x_data = x_data[:num_users - num_users % batch_size]
    y_data = y_data[:num_users - num_users % batch_size]

    return x_data, y_data

In [1]:
def preprocess(input_file, y_date, seq_len=16, batch_size=32, exclude_date=None, d_model=35):
    """
    Preprocess data and split it in train and test data
    :param d_model:
    :param input_file: string, path to raw dataset, csv file
    :param y_date: string, timestamp use for testing
    :param exclude_date: list[string] timestamps to ignore
    :return: train_x, train_y (both are np.array)
    """
    months_one_hot = [0 for _ in range(12)]
    segmentation_dict = {}
    x_users, y_users = {}, {}
    df = pd.read_csv(input_file)
    df = clean_data(df)
    users = []
    for i, row in df.iterrows():
        if row['fecha_dato'] in exclude_date:
            pass
        user = row['ncodpers'] # uid
        date = row['fecha_dato'].split("-")
        year = [int(date[0] == "2016")]  # 1=2016, 0=2015 (1)
        month = copy.copy(months_one_hot)
        month[int(date[1]) - 1] = 1  # months one-hot encoded (12)
        items = list(row.values)[26:]  # items are one-hot encoded (22)
        items = [int(item) if not math.isnan(item) and item != 'NA' else 0 for item in items]
        # one-hot encode segmentation (4)
        segmentation = row['segmento']
        segmentation_array = [0, 0, 0, 0]
        if segmentation not in segmentation_dict.keys():
            segmentation_dict[segmentation] = len(segmentation_dict)
        segmentation_array[segmentation_dict[segmentation]] = 1
        # one-hot encode new-index (1)
        #new_index = [1] if row['ind_nuevo'] == 1 else [0]
        # seniority + age + income (3) - values features
        seniority = float(row['antiguedad'])
        age = float(row['age'])
        income = float(row['renta'])
        value_features = [seniority, age, income]
        # put the data together
        data = year + month + segmentation_array + value_features + items  # (42) values
        if row['fecha_dato'] == y_date and user in x_users.keys():
            y_value = [0]
            if sum(items) > 0:
                y_value = [1]
            y_users[user] = y_value
            users.append(user)
        elif user in x_users.keys():
            x_users[user] = np.vstack((x_users[user], np.array(data)))
        else:
            x_users[user] = np.array(data)

    assert len(x_users) == len(y_users)
    x_data = []
    y_data = []
    for user in users:
        if np.array(x_users[user]).shape[0] == seq_len:
            x_data.append(x_users[user].reshape((seq_len, d_model)))
        else:
            continue
        y_data.append(y_users[user])
    x_data = np.stack(x_data)
    y_data = np.stack(y_data)
    num_users = x_data.shape[0]
    x_data = x_data[:num_users - num_users % batch_size]
    y_data = y_data[:num_users - num_users % batch_size]

    return x_data, y_data

In [31]:
args_dataset = 'data/train_reduced.csv'
args_seq_len = 16
args_batch_size = 64
args_d_model = 42
x_train, y_train = preprocess(args_dataset, y_date="2016-04-28", exclude_date=["2016-05-28"], seq_len=args_seq_len, batch_size=args_batch_size, d_model=args_d_model)

C:\Users\oldwhite\AppData\Local\Temp\ipykernel_47032\3116379530.py:13: DtypeWarning: Columns (10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


In [32]:
x_train.shape

(19968, 16, 42)

In [34]:
y_train[y_train==0].shape

(686,)

In [35]:
y_train[y_train==1].shape

(19282,)

In [18]:
x_train[0].shape

(16, 42)

# y_train
multilabel classification

In [7]:
y_train.shape

(19968, 22)

In [8]:
y_train[0]

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])

# evaluate_one_epoch debugging

In [76]:
import torch
from tqdm import tqdm
def evaluate_one_epoch(model, criterion, dataset, device="cpu", owned_items=None):
    batch_size = 1
    generator = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size
    )
    model.eval()
    tot_loss = 0.0
    tot_prec1 = 0.0
    n_users = 0
    j = 0
    with torch.no_grad():
        for batch, labels in tqdm(generator):
            batch, labels = batch.to(device), labels.to(device)
            print(labels)
            logits = model(batch)
            #print('prediction: ' + str(logits))
            #print('labels: ' + str(labels))
            #print(labels.shape)
# Comment temporarily
#             if labels.size(1) != 1:
#                 print(labels.shape)
#                 continue
#            loss = criterion(logits, labels)
#            tot_loss += loss.item()
#            recommendations = logits_to_recs(logits.detach().cpu().numpy())
#            tot_prec1 += precision_k(1, labels, recommendations)
#            n_users += 1 # not sure
        tot_loss /= len(dataset) // batch_size
        tot_prec1 /= n_users
        metrics_dict = {"prec1": tot_prec1}
    return tot_loss, metrics_dict

In [51]:
def logits_to_recs(logits):
    logits = np.squeeze(logits)
    recs = np.argsort(logits)[::-1]
    return recs

In [54]:
def precision_k(k, gt, preds):
    """
    :param k: int, scope of metric
    :param gt: list[int], index of ground truth recommendations
    :param preds: list[int], index of predicted recommendations
    """
    c = 0
    for p in preds[:k]:
        if p in gt:
            c += 1
    return c / k

In [39]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, train_x, train_y, nrows=None):
        if nrows is None:
            self.data = [(x, y) for x, y in zip(train_x, train_y)]
        else:
            self.data = [(x, y) for x, y in zip(train_x[:nrows], train_y[:nrows])]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x, y = self.data[index]
        x = torch.FloatTensor(x)
        y = torch.FloatTensor(y)
        return x, y

x_test, y_test = preprocess(args_dataset, y_date="2016-05-28", exclude_date=["2015-01-28"], seq_len=args_seq_len, batch_size=args_batch_size, d_model=args_d_model)
test_set = CustomDataset(x_test, y_test, nrows=None)

C:\Users\oldwhite\AppData\Local\Temp\ipykernel_47032\3116379530.py:13: DtypeWarning: Columns (10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


In [49]:
from model.transformer import get_model
import torch.nn as nn
criterion = nn.BCEWithLogitsLoss()
args_n_items = 1 # not very sure
args_d_model = 42
args_heads = 7
args_dropout = 0.5
args_n_layers = 6
args_hidden_size = 2048
model = get_model(args_n_items, args_d_model, args_heads, args_dropout, args_n_layers, args_hidden_size, None, 'cpu')

In [82]:
y_test[:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [77]:
test_loss, test_metrics = evaluate_one_epoch(model, criterion, test_set, 'cpu', owned_items=None)

  0%|          | 26/19968 [00:00<01:19, 252.11it/s]

tensor([[0.]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[42.]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[5.8855e-44]])
tensor([[-3.0663e-19]])
tensor([[-4.2514e-20]])
tensor([[-3.0663e-19]])
tensor([[nan]])
tensor([[-4.2514e-20]])
tensor([[42.0000]])
tensor([[16.]])
tensor([[-3.0663e-19]])
tensor([[-4.2514e-20]])
tensor([[1.4013e-45]])
tensor([[-4.2514e-20]])
tensor([], size=(1, 0))
tensor([[-0.8242]])
tensor([[9.9838e-07]])
tensor([[-4.2514e-20]])
tensor([[42.]])
tensor([[6.9167e-27]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[42.0000]])
tensor([[1.4013e-45]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[42.]])
tensor([[1.4013e-45]])
tensor([[16.]])
tensor([[-4.2514e-20]])
tensor([[4.2352e-20]])
tensor([[2.4375]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[1.0000e-06]])
tensor([[-4.2514e-20]])
tensor([[1.0000e-06]])
tensor([[-3.0663e-19]])
tensor([[42.0000]])
tensor([[9.9838e-07]])
tensor([

  0%|          | 79/19968 [00:00<01:16, 259.27it/s]

tensor([[1.4013e-45]])
tensor([[-3.0663e-19]])
tensor([[-3.0663e-19]])
tensor([[1.4013e-45]])
tensor([[-4.2514e-20]])
tensor([[1.0000e-06]])
tensor([[1.4013e-45]])
tensor([[1.4013e-45]])
tensor([[-4.2514e-20]])
tensor([[1.4013e-45]])
tensor([[42.0000]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[42.0000]])
tensor([[2.4375]])
tensor([], size=(1, 0))
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[-3.0663e-19]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[2.4375]])
tensor([[5.8855e-44]])
tensor([[-4.2514e-20]])
tensor([[2.4375]])
tensor([[1.0000e-06]])
tensor([[9.9838e-07]])
tensor([[42.0000]])
tensor([[1.4013e-45]])
tensor([[2.4375]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[1.4013e-45]])
tensor([], size=(1, 0))
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[4.2352e-20]])
tensor([[1.4013e-45]])
tensor([[0.]])
tensor([[9.9838e-07]])

  1%|          | 134/19968 [00:00<01:15, 261.18it/s]

...

 10%|▉         | 1976/19968 [00:07<01:12, 248.07it/s]

tensor([[-4.2514e-20]])
tensor([[6.9167e-27]])
tensor([[-4.2514e-20]])
tensor([[1.4013e-45]])
tensor([[-3.0663e-19]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[6.9233e-27]])
tensor([[-4.2514e-20]])
tensor([[42.]])
tensor([[-3.0663e-19]])
tensor([[-3.0663e-19]])
tensor([[1.0000e-06]])
tensor([[0.1581]])
tensor([[9.9838e-07]])
tensor([[9.9838e-07]])
tensor([[1.0000e-06]])
tensor([[-4.2514e-20]])
tensor([[-3.0820e-19]])
tensor([[-4.2514e-20]])
tensor([[42.0000]])
tensor([[-0.9175]])
tensor([[9.9838e-07]])
tensor([[-3.0820e-19]])
tensor([[-3.0663e-19]])
tensor([[-4.2514e-20]])
tensor([[-4.2514e-20]])
tensor([[42.0000]])
tensor([[-3.0820e-19]])
tensor([[6.9167e-27]])
tensor([[5.8855e-44]])
tensor([[1.0000e-06]])
tensor([[-4.2514e-20]])
tensor([[1.4013e-45]])
tensor([[-3.0820e-19]])
tensor([], size=(1, 0))
tensor([[2.4375]])
tensor([[9.9838e-07]])
tensor([[6.9167e-27]])
tensor([[-3.0820e-19]])
tensor([[42.0000]])
tensor([[1.4013e-45]])
tensor([[1.4013e-45]])
tensor([[16.]])
te

 10%|█         | 2002/19968 [00:07<01:09, 257.22it/s]

tensor([[-4.2514e-20]])
tensor([[6.9233e-27]])
tensor([[0.]])
tensor([[2.4495]])
tensor([], size=(1, 0))
tensor([[1.4013e-45]])
tensor([[-4.2514e-20]])
tensor([[0.]])
tensor([[1.4013e-45]])
tensor([[-4.2514e-20]])
tensor([[1.4013e-45]])
tensor([[42.0000]])
tensor([[1.0000e-06]])
tensor([[1.0000e-06]])
tensor([], size=(1, 0))
tensor([[-4.2514e-20]])
tensor([[-3.0663e-19]])
tensor([[1.0000e-06]])
tensor([[-4.2514e-20]])


KeyboardInterrupt: 